# Import Packages

In [1]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import os

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Import Data

In [3]:
train_data = np.genfromtxt('data/MNIST/train.csv', delimiter=',', skip_header=1)
test_data = np.genfromtxt('data/MNIST/test.csv', delimiter=',', skip_header=1)

In [4]:
(train_data.shape, test_data.shape)

((42000, 785), (28000, 784))

In [5]:
X, y =  train_data[:,1:].copy(), train_data[:,0].copy() # features, labels

In [6]:
X_test = test_data.copy()

In [7]:
(X.shape, y.shape, X_test.shape)

((42000, 784), (42000,), (28000, 784))

# Simple Pre-Process Data

In [8]:
# Normalization
X /= 255.0
X_test /=  255.0

# One Hot Encoding
NUM_DIGITS = 10
y = tf.keras.utils.to_categorical(y, NUM_DIGITS)

In [9]:
# Splitting data to training and validation (optional as model.fit allows for validation split as well)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=42)

In [10]:
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape, X_test.shape

((31500, 784), (10500, 784), (31500, 10), (10500, 10), (28000, 784))

In [11]:
# Some constants
TRAINING_SIZE = len(X_train)
VALIDATION_SIZE = len(X_valid)
TEST_SIZE = len(X_test)
IMAGE_SIZE_UNSTACKED = X_train.shape[1]

TRAINING_SIZE, VALIDATION_SIZE, TEST_SIZE, IMAGE_SIZE_UNSTACKED

(31500, 10500, 28000, 784)

# Build NN Model

In [12]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(IMAGE_SIZE_UNSTACKED,)))
model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))

In [13]:
optimizer = tf.train.RMSPropOptimizer(learning_rate=0.001)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


# Fitting Model with training data

## Method 1 with built-in validation split

In [14]:
model.fit(X, y, epochs=5, validation_split=0.25)

Train on 31500 samples, validate on 10500 samples
Epoch 1/5
31500/31500 [==============================] - 6s 182us/step - loss: 0.4229 - acc: 0.8684 - val_loss: 0.1663 - val_acc: 0.9510
Epoch 2/5
31500/31500 [==============================] - 4s 122us/step - loss: 0.1165 - acc: 0.9661 - val_loss: 0.1132 - val_acc: 0.9650
Epoch 3/5
31500/31500 [==============================] - 4s 120us/step - loss: 0.0776 - acc: 0.9777 - val_loss: 0.0905 - val_acc: 0.9739
Epoch 4/5
31500/31500 [==============================] - 4s 115us/step - loss: 0.0552 - acc: 0.9837 - val_loss: 0.1017 - val_acc: 0.9706
Epoch 5/5
31500/31500 [==============================] - 4s 118us/step - loss: 0.0410 - acc: 0.9875 - val_loss: 0.1000 - val_acc: 0.9738


## Method 2 with manual validation split

In [15]:
model.fit(X_train, y_train, epochs=5, validation_data=(X_valid, y_valid))

Train on 31500 samples, validate on 10500 samples
Epoch 1/5
31500/31500 [==============================] - 3s 108us/step - loss: 0.0515 - acc: 0.9852 - val_loss: 0.0446 - val_acc: 0.9882
Epoch 2/5
31500/31500 [==============================] - 3s 105us/step - loss: 0.0356 - acc: 0.9895 - val_loss: 0.0482 - val_acc: 0.9864
Epoch 3/5
31500/31500 [==============================] - 3s 107us/step - loss: 0.0284 - acc: 0.9916 - val_loss: 0.0636 - val_acc: 0.9855
Epoch 4/5
31500/31500 [==============================] - 3s 109us/step - loss: 0.0206 - acc: 0.9940 - val_loss: 0.0585 - val_acc: 0.9863
Epoch 5/5
31500/31500 [==============================] - 3s 108us/step - loss: 0.0158 - acc: 0.9951 - val_loss: 0.0588 - val_acc: 0.9862


# Save Model

In [16]:
os.makedirs('models', exist_ok=True)
os.makedirs('submissions', exist_ok=True)

In [17]:
tf.keras.models.save_model(
    model,
    "models/digit-recognizer-TF-Keras-NN-Basic",
    include_optimizer=False
)

# Load Model

In [18]:
md = tf.keras.models.load_model("models/digit-recognizer-TF-Keras-NN-Basic", compile=False)

# Make Prediction on Test Data

In [21]:
prediction = md.predict(X_test)

In [22]:
label = tf.argmax(prediction,axis=1)
image_id = np.array(np.arange(1,TEST_SIZE+1))

## Convert result Tensor back to Numpy Array

In [23]:
from keras import backend as K

np_label = K.eval(label)
result = np.column_stack((image_id, np_label))

Using TensorFlow backend.


## Save results as csv

In [24]:
np.savetxt("submissions/digit-recognizer-TF-Keras-NN-Basic.csv", result, fmt=('%d,%d'), delimiter=",", \
           header="ImageId,Label")

!kaggle competitions submit -c digit-recognizer -f output/digit-recognizer-TF-Keras-NN-Basic.csv -m "TF/Keras NN Basic"